In [1]:
import os
import sys
import os
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["TF_NUM_INTRAOP_THREADS"] = "4"
os.environ["TF_NUM_INTEROP_THREADS"] = "2"

import tensorflow as tf
tf.keras.backend.set_floatx("float32")


# Add SRC to path
sys.path.append("/Users/nedim/Desktop/MY_REPOS/mids-w207-section1-team1-finalproject/src")

from data_prep import load_and_prepare_data
from modeling import build_fnn, train_fnn, evaluate_fnn
import numpy as np

2025-12-07 16:14:05.369953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Check tf version due to my intel mac, needs to be 2.11 per tensorflow documentation
print("TF VERSION:", tf.__version__)

TF VERSION: 2.11.0


In [3]:
csv_path = "../data/external/FPA_FOD_Plus.csv"

print("Loading and preparing data...")
X_train, Y_train, X_val, Y_val, _, _ = load_and_prepare_data(
    csv_path,
    smote_strategy="smotenc"
)

print("\nFull Training Shape:", X_train.shape)
print("Full Validation Shape:", X_val.shape)

Loading and preparing data...

Loading data with Polars...
File loaded into Pandas. Shape: (2302521, 308)

Pre-Sampling before SMOTENC (500k small class)...

Original class distribution:
target
0    1345082
1      33560
2       2210
3        660
Name: count, dtype: int64

Pre-sampled total rows: 536,430

Applying SMOTENC oversampling

Full Training Shape: (2000000, 49)
Full Validation Shape: (460504, 49)


In [4]:
SAMPLE_SIZE = 500_000
rng = np.random.default_rng(seed=42)
idx = rng.choice(len(X_train), size=SAMPLE_SIZE, replace=False)

X_train_big = X_train[idx]
Y_train_big = Y_train[idx]

print("Training on:", X_train_big.shape, Y_train_big.shape)

Training on: (500000, 49) (500000,)


In [5]:
# Build FNN Model
print("\nBuilding FNN Model...")

model = build_fnn(
    input_dim=X_train_big.shape[1],
    hidden_layers=[64, 32],
    dropout_rate=0.30,
    learning_rate=0.005
)



Building FNN Model...


2025-12-07 16:24:47.472904: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
print("Training on:", X_train_big.shape, Y_train_big.shape)

Training on: (500000, 49) (500000,)


In [7]:
model, history = train_fnn(
    model,
    X_train_big,
    Y_train_big,
    X_val,
    Y_val,
    epochs=20,
    batch_size=128
)

Epoch 1/20
3907/3907 [==============================] - 12s 3ms/step - loss: 0.8760 - accuracy: 0.5989 - val_loss: 0.6229 - val_accuracy: 0.7722
Epoch 2/20
3907/3907 [==============================] - 11s 3ms/step - loss: 0.8101 - accuracy: 0.6358 - val_loss: 0.6667 - val_accuracy: 0.6969
Epoch 3/20
3907/3907 [==============================] - 12s 3ms/step - loss: 0.7862 - accuracy: 0.6484 - val_loss: 0.6290 - val_accuracy: 0.7335
Epoch 4/20
3907/3907 [==============================] - 11s 3ms/step - loss: 0.7739 - accuracy: 0.6553 - val_loss: 0.6725 - val_accuracy: 0.6909


In [8]:
# Validation Metrics Only
print("\nVALIDATION METRICS")
evaluate_fnn(model, X_val, Y_val)


VALIDATION METRICS
14391/14391 [==============================] - 15s 1ms/step

Validation Accuracy: 0.7722473637579695

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.79      0.87    448361
           1       0.05      0.26      0.09     11185
           2       0.01      0.37      0.02       738
           3       0.01      0.52      0.02       220

    accuracy                           0.77    460504
   macro avg       0.26      0.49      0.25    460504
weighted avg       0.96      0.77      0.85    460504


Confusion Matrix:
 [[352276  53986  28840  13259]
 [  5115   2959   2027   1084]
 [   134    105    273    226]
 [    12     15     78    115]]


array([0, 0, 0, ..., 2, 2, 2])